In [1]:
!kaggle competitions download -c ss3-final-iot-farm-connect

  0%|                                               | 0.00/37.7k [00:00<?, ?B/s]
100%|███████████████████████████████████████| 37.7k/37.7k [00:00<00:00, 842kB/s]


In [2]:
!unzip ss3-final-iot-farm-connect.zip

Archive:  ss3-final-iot-farm-connect.zip
  inflating: sample_submission.csv   
  inflating: sensors_info.csv        
  inflating: test/set02.csv          
  inflating: test/set09.csv          
  inflating: train/set01.csv         
  inflating: train/set03.csv         
  inflating: train/set04.csv         
  inflating: train/set05.csv         
  inflating: train/set06.csv         
  inflating: train/set07.csv         
  inflating: train/set08.csv         
  inflating: train/set10.csv         
  inflating: train/set11.csv         
  inflating: train/set12.csv         


In [4]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import os

In [26]:
t1 = pd.read_csv("train/set01.csv")
t2 = pd.read_csv("train/set03.csv")
t3 = pd.read_csv("train/set04.csv")
t4 = pd.read_csv("train/set05.csv")
t6 = pd.read_csv("train/set06.csv")
t7 = pd.read_csv("train/set07.csv")
t8 = pd.read_csv("train/set08.csv")
t10 = pd.read_csv("train/set10.csv")
t11 = pd.read_csv("train/set11.csv")
t12 = pd.read_csv("train/set12.csv")

In [27]:
df = pd.concat([t1, t2, t3, t4, t6, t7, t8, t10, t11, t12], ignore_index=True)

In [29]:
df = df.dropna()

In [30]:
df.isna().sum()

index                     0
time                      0
1Air Humidity             0
1AirTemperature           0
1LightHigh                0
1LightLow                 0
11Rain Accumulate (mm)    0
11Rain count              0
12WindSpeed               0
13WindDirection           0
23SoilWC ค.ชื้นดิน        0
23SoilTemp.อุณหภูมิดิน    0
23SoilEC                  0
23SoilpH                  0
26N                       0
26P                       0
26K                       0
29SoilTension             0
flow1                     0
dtype: int64

In [31]:
df

,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1
0,01_1,01:00:50.174233+00,97.3,21.8,0.0,9.57,0.0,0.0,0.0,6.0,31.4,23.6,0.11,8.3,21.0,95.0,87.0,23.0,0.0
1,01_2,01:02:51.384714+00,96.6,22.0,0.0,9.81,0.0,0.0,0.0,6.0,31.9,23.6,0.11,8.3,21.0,95.0,87.0,22.0,0.0
3,01_4,01:06:50.205974+00,95.3,22.4,0.0,10.25,0.0,0.0,0.0,6.0,32.4,23.6,0.11,8.3,21.0,95.0,87.0,22.0,0.0
4,01_5,01:08:50.162332+00,93.8,22.4,0.0,10.45,0.0,0.0,0.0,6.0,31.7,23.6,0.11,8.3,21.0,95.0,87.0,23.0,0.0
5,01_6,01:10:50.226361+00,95.9,22.8,0.0,10.57,0.0,0.0,0.0,2.0,31.4,23.5,0.11,8.3,21.0,95.0,87.0,23.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,12_108,04:46:16.11276+00,45.9,32.0,0.0,50.06,0.0,0.0,0.6,1.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,110.0,0.0
1105,12_109,04:48:16.120498+00,41.4,33.1,0.0,50.36,0.0,0.0,0.7,7.0,37.6,22.8,0.11,5.0,21.0,95.0,87.0,108.0,0.0
1106,12_110,04:50:16.123489+00,43.0,33.0,0.0,52.30,0.0,0.0,1.0,5.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,109.0,0.0
1108,12_112,04:54:16.141528+00,41.7,33.1,0.0,54.86,0.0,0.0,0.8,4.0,37.4,22.8,0.11,5.1,21.0,95.0,87.0,109.0,0.0


In [34]:
df["time"] = df["time"].astype("datetime64[ns]")

/tmp/ipykernel_12304/2605481861.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time"] = df["time"].astype("datetime64[ns]")


In [37]:
df

,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1
0,01_1,2023-03-25 01:00:50.174233,97.3,21.8,0.0,9.57,0.0,0.0,0.0,6.0,31.4,23.6,0.11,8.3,21.0,95.0,87.0,23.0,0.0
1,01_2,2023-03-25 01:02:51.384714,96.6,22.0,0.0,9.81,0.0,0.0,0.0,6.0,31.9,23.6,0.11,8.3,21.0,95.0,87.0,22.0,0.0
3,01_4,2023-03-25 01:06:50.205974,95.3,22.4,0.0,10.25,0.0,0.0,0.0,6.0,32.4,23.6,0.11,8.3,21.0,95.0,87.0,22.0,0.0
4,01_5,2023-03-25 01:08:50.162332,93.8,22.4,0.0,10.45,0.0,0.0,0.0,6.0,31.7,23.6,0.11,8.3,21.0,95.0,87.0,23.0,0.0
5,01_6,2023-03-25 01:10:50.226361,95.9,22.8,0.0,10.57,0.0,0.0,0.0,2.0,31.4,23.5,0.11,8.3,21.0,95.0,87.0,23.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,12_108,2023-03-25 04:46:16.112760,45.9,32.0,0.0,50.06,0.0,0.0,0.6,1.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,110.0,0.0
1105,12_109,2023-03-25 04:48:16.120498,41.4,33.1,0.0,50.36,0.0,0.0,0.7,7.0,37.6,22.8,0.11,5.0,21.0,95.0,87.0,108.0,0.0
1106,12_110,2023-03-25 04:50:16.123489,43.0,33.0,0.0,52.30,0.0,0.0,1.0,5.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,109.0,0.0
1108,12_112,2023-03-25 04:54:16.141528,41.7,33.1,0.0,54.86,0.0,0.0,0.8,4.0,37.4,22.8,0.11,5.1,21.0,95.0,87.0,109.0,0.0


In [38]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="index",
    timestamp_column="time"
)
train_data.head()

,,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1
item_id,timestamp,,,,,,,,,,,,,,,,,
01_1,2023-03-25 01:00:50.174233,97.3,21.8,0.0,9.57,0.0,0.0,0.0,6.0,31.4,23.6,0.11,8.3,21.0,95.0,87.0,23.0,0.0
01_2,2023-03-25 01:02:51.384714,96.6,22.0,0.0,9.81,0.0,0.0,0.0,6.0,31.9,23.6,0.11,8.3,21.0,95.0,87.0,22.0,0.0
01_4,2023-03-25 01:06:50.205974,95.3,22.4,0.0,10.25,0.0,0.0,0.0,6.0,32.4,23.6,0.11,8.3,21.0,95.0,87.0,22.0,0.0
01_5,2023-03-25 01:08:50.162332,93.8,22.4,0.0,10.45,0.0,0.0,0.0,6.0,31.7,23.6,0.11,8.3,21.0,95.0,87.0,23.0,0.0
01_6,2023-03-25 01:10:50.226361,95.9,22.8,0.0,10.57,0.0,0.0,0.0,2.0,31.4,23.5,0.11,8.3,21.0,95.0,87.0,23.0,0.0


In [39]:
train_data.columns

Index(['1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow',
       '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed',
       '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน',
       '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1'],
      dtype='object')

In [42]:
import sklearn.metrics
from autogluon.core.metrics import make_scorer

In [64]:
predictor = TimeSeriesPredictor(
    prediction_length=48,
    path="autogluon-m4-hourly2",
    target="23SoilWC ค.ชื้นดิน",
    ignore_time_index=True,
    eval_metric="sMAPE",
)

predictor.fit(
    train_data
)

/home/superai023/.conda/envs/newpt/lib/python3.9/site-packages/autogluon/timeseries/predictor.py:248: UserWarning: Detected time series with length <= 2 in data. Please remove them from the dataset or TimeSeriesPredictor likely won't work as intended.
  warnings.warn(
/home/superai023/.conda/envs/newpt/lib/python3.9/site-packages/autogluon/timeseries/predictor.py:436: UserWarning: Detected short time series in train_data. For best performance, all training time series should have length >= 2 * prediction_length + 1(at least 97).
  warnings.warn(
================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'sMAPE',
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'prediction_length': 48,
 'random_seed': None,
 'target': '23SoilWC ค.ชื้นดิน',
 'time_limit': None}
Provided training data set with 926 rows, 926 items (item = single time series). Average time series length is

ValueError: Cannot create a validation set because all training time series are too short. At least some time series in train_data must have length >= 2 * prediction_length + 1 (at least 97) but the longest training series has length 1. Please decrease prediction_length, provide longer time series in train_data, or provide tuning_data.